# Loading and cleaning

In [1]:
import pandas as pd
import numpy as np
import glob
import re

In [2]:
files = [f for f in glob.glob("../repository/RAW/BTI*")]
dfs = []
for i, f in enumerate(files):
    year = re.search("(\d{4})", f).group(0)
    with open(f, mode="r", encoding="utf-8"):
        _df = pd.read_csv(f, sep=";", index_col=False, decimal=',')
        regions = {
            k.strip(): v.strip()
            for k, v in [el.split("|") for el in _df.columns[0].split("\n")[1:]]
        }
        _df = _df.rename(str.strip, axis="columns")
        _df = _df.dropna(axis=1, how="all")
        _df["Region"] = _df["Region"].astype(str)
        _df["Region"] = _df["Region"].replace(regions)
        _df = _df.rename({_df.columns[0]: "Country"}, axis="columns")
        _df["Year"] = year
        _df = _df.replace(['-', '?'], np.nan)
        _df = _df.convert_dtypes()
        _df = _df[[*_df.columns[:2], "Year", *_df.columns[2:]]] # Reorder Year column
        _df = _df.loc[:,~_df.columns.duplicated()].copy() # Remove Year duplicates
        to_drop = []
        ptrn = re.compile('Trend.*')
        for i, col in enumerate(_df.columns):
            is_rm = re.match(ptrn, col)
            if is_rm:
                to_drop.append(i + 1) 
            try:
                _df[col] = _df[col].str.replace(',', '.').astype(float)
                try:
                    _df[col] = _df[col].astype(int)
                except ValueError:
                    pass
            except (ValueError, AttributeError):
                continue
        _df = _df.drop([_df.columns[i] for i in to_drop], axis=1) # Remove trending arrows
        dfs.append(_df)

In [3]:
cols = dfs[1].columns
for df in dfs[1:]:
    df.columns = cols
df = pd.concat([*dfs[1:]])


In [4]:
# df.to_csv('../repository/DUMP/Trasnformation Index (2008-2022)', sep=';')

# Categorizing

In [5]:
df['Q2.1 | Free and fair elections']

0       6.0
1       8.0
2       4.0
3       6.0
4      10.0
       ... 
132     2.0
133     2.0
134     1.0
135     5.0
136     3.0
Name: Q2.1 | Free and fair elections, Length: 959, dtype: Float64